In [ ]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

In [ ]:
!pip install -U sagemaker

In [ ]:
import sagemaker.huggingface

In [ ]:
import sagemaker
import logging
import sys

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
def setup_sagemaker_session(logger=None):
    # 1. Get Sagemaker session
    # 2. Get default bucket
    # 3. Get sagemaker execution role

    sagemaker_session = sagemaker.Session()
    bucket = sagemaker_session.default_bucket()
    role = sagemaker.get_execution_role()
    
    if logger is None:
        print(f"sagemaker role arn: {role}")
        print(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
        print(f"sagemaker session region: {sagemaker_session.boto_region_name}")
    else:
        logger.info(f"sagemaker role arn: {role}")
        logger.info(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
        logger.info(f"sagemaker session region: {sagemaker_session.boto_region_name}")

    return sagemaker_session, bucket, role

In [ ]:
sm_sess, bucket, role = setup_sagemaker_session(logger)

In [ ]:
!git pull origin main

In [ ]:
from sagemaker.huggingface import HuggingFace

In [ ]:
hyperparameters={
    'epochs': 10,
     'batch_size': 32,
     'model':'bert-based-uncased'
}
base_job_prefix = "huggingface-classification-arxiv"
model_path = f"s3://{sm_sess.default_bucket()}/{base_job_prefix}/model"

In [ ]:
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./src',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    transformers_version='4.12',
    pytorch_version='1.9',
    py_version='py38',
    output_path=model_path,
    hyperparameters=hyperparameters
)

In [ ]:
huggingface_estimator.fit()